# Implementing Huffman Coding

In [47]:
import heapq
import os

class HuffmanCoding:
    
    def __init__(self,path):
        self.path =  path
        self.__heap = []
        self.__codes = {}
        self.__reverseCodes = {}
    
    def __make_frequency_dict(self,text):
        freq = {}
        for char in text:
            freq[char] = freq.get(char,0)+1
        return freq
    
    def __buildHeap(self,freq_dict):
        for key in freq_dict:
            frequency = freq_dict[key]
            binary_tree_node = BinaryTreeNode(key,frequency)
            heapq.heappush(self.__heap,binary_tree_node)
        
    def __buildTree(self):
        
        while (len(self.__heap) > 1):
            
            binary_tree_node_1 = heapq.heappop(self.__heap)
            binary_tree_node_2 = heapq.heappop(self.__heap)
            freq_sum = binary_tree_node_1.freq + binary_tree_node_2.freq
            newNode = BinaryTreeNode(None,freq_sum)
            newNode.left = binary_tree_node_1
            newNode.right = binary_tree_node_2
            heapq.heappush(self.__heap,newNode)
        return 
    
    def __buildCodesHelper(self,root,curr_bits):
        
        if root is None:
            return
        if root.value is not None:
            self.__codes[root.value] = curr_bits
            self.__reverseCodes[curr_bits] = root.value
            return
        
        self.__buildCodesHelper(root.left,curr_bits+"0")
        self.__buildCodesHelper(root.right,curr_bits+"1")
    
    def __buildCodes(self):
        
        root = heapq.heappop(self.__heap)
        self.__buildCodesHelper(root,"")
        
    
    def __getEncodedtext(self,text):
        encoded_text = ""
        
        for char in text:
            encoded_text += self.__codes[char]
        return encoded_text
    
    def __getPaddedEncodedText(self,encoded_text):
        
        padded_amount = 8-len(encoded_text) % 8
        
        for i in range(padded_amount):
            encoded_text+='0'
        padded_info = "{0:08b}".format(padded_amount)
        padded_encoded_text = padded_info+encoded_text
        return padded_encoded_text
    
    def __getBytesArray(self,padded_encoded_text):
        bytes_array = []
        for i in range(0,len(padded_encoded_text),8):
            byte_form = int(padded_encoded_text[i:i+8],2)
            bytes_array.append(byte_form)
        return bytes_array
        
    
    def compress(self):
        # get file from path
        # read text from file
        file_name,file_extension = os.path.splitext(self.path)
        output_path = file_name + '.bin'
        
        with open(self.path,'r+') as file , open(output_path,'wb') as output:
        
            # make frequency dict using text

            text = file.read()
            text = text.rstrip()
            
            freq_dict = self.__make_frequency_dict(text)

            # Construct the heap from frequency_dictionary

            self.__buildHeap(freq_dict)

            # Construct the binary tree from heap
            self.__buildTree()

            # Constructing Codes from Binary Tree

            self.__buildCodes()

            # Encoding text using Codes

            encoded_text = self.__getEncodedtext(text)

            # pad this encoded text

            padded_encoded_text = self.__getPaddedEncodedText(encoded_text)

            # Store as bytes

            bytes_array = self.__getBytesArray(padded_encoded_text)

            final_bytes = bytes(bytes_array)
            
            output.write(final_bytes)
            print("Compressed ...")
        return output_path
    
    def __removePadding(self,bit_string):
        padding_info = bit_string[:8]
        extra_padding = int(padding_info,2)
        text = bit_string[8:]
        text_after_padding_removed = text[:-extra_padding]
        return text_after_padding_removed
        
    def __decodeText(self,actual_text):
        decoded_text =''
        current_bits = ''
        for bit in actual_text:
            current_bits += bit
            if current_bits in self.__reverseCodes:
                character = self.__reverseCodes[current_bits]
                decoded_text += character
                current_bits = ''
        return decoded_text
            
    def decompress(self,input_path):
        filename,file_extension = os.path.splitext(self.path)
        output_path = filename + "_decompressed" + '.txt'
        
        with open(input_path,'rb') as file , open(output_path,'w') as output:
            bit_string = ''
            byte = file.read(1)
            while byte:
                byte = ord(byte)
                bits = bin(byte)[2:].rjust(8,'0')
                bit_string += bits
                byte = file.read(1)
            
            
            actual_text = self.__removePadding(bit_string)
            decompress_text = self.__decodeText(actual_text)
            print("Decompressed ...")
            output.write(decompress_text)
            

                             
class BinaryTreeNode:
    
    def __init__(self,value,freq):
        
        self.value = value
        self.freq = freq
        self.left = None
        self.right = None
        
    def __lt__(self,other):
        self.freq < other.freq
        
    def __eq__(self,other):
        self.freq == other.freq

In [48]:
path = 'sample.txt'
h = HuffmanCoding(path)
output_path = h.compress()
print(output_path)
decompressed_output_path = h.decompress(output_path)

Compressed ...
sample.bin
Decompressed ...
